<a href="https://colab.research.google.com/github/jshogland/SpatialModelingTutorials/blob/main/Notebooks/GetRestData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting data from a rest api and open streatmaps
In this example we will download data from ESRI, Landfire, USFS image and feature services for the extent of the Stanislaus National Forest. We will also be getting date from Open Street Maps.

#### author John Hogland

install packages

In [ ]:
!pip install --upgrade gdown
!pip install --upgrade numba
!pip install --upgrade geopandas
!pip install mapclassify
!pip install --upgrade datascience
!pip install --upgrade gym
!pip install --upgrade folium
!pip install localtileserver
!pip install osmnx
!pip install raster_tools

import modules

In [ ]:
from raster_tools import Raster, distance, general, Vector, open_vectors, clipping, surface, creation, focal
import numpy as np, geopandas as gpd, pandas as pd, osmnx as ox, numba as nb
import shapely, requests, urllib.request, os

## Get the boundary and buffer of the Stanislaus National Forest
Visualize the boundary

In [ ]:
anf=ox.geocode_to_gdf('Stanislaus National Forest, CA, USA')
ply=anf.buffer(0.25)
ply.explore()

## Get Image Service Data
Create methods for getting image and featrue layer services data.

In [ ]:
#definition to extract data from REST service
def get_image_service_data(url, ply, out_prefix,res=30,outSR=""):
    '''
    extracts a list of images from a image service given a url, polygon, and output prefix name

    url = (string) path to image service e.g., url=r'https://lfps.usgs.gov/arcgis/rest/services/Landfire_LF230/US_230EVT/ImageServer'
    ply = (geoseries or geodataframe) of the study area
    out_prefix = (string) prefix used to save each image
    '''
    layerInfo=requests.get(url+'?f=pjson')
    dic=layerInfo.json()
    #print(dic)
    spr=dic['spatialReference']
    m_width=dic['maxImageWidth']
    m_height=dic['maxImageHeight']
    fitem=next(iter(spr))
    ply2=ply.to_crs(spr[fitem])

    xmin,ymin,xmax,ymax=ply2.total_bounds

    wcells=int((xmax-xmin)/res)
    hcells=int((ymax-ymin)/res)

    if(wcells<m_width):
        m_width=wcells

    if(hcells<m_height):
        m_height=hcells


    wcells_l=np.arange(0,wcells,m_width)
    hcells_l=np.arange(0,hcells,m_height)

    xmax2=xmin
    ymax2=ymin

    tile=1

    rs_lst=[]
    for w in wcells_l:
        for h in hcells_l:
            xmax2 = (m_width*res+xmax2)
            ymax2 = (m_height*res+ymax2)

            qry = url+'/exportImage?'
            parm = {
                'f':'json',
                'bbox':','.join([str(xmin),str(ymin),str(xmax2),str(ymax2)]),
                'size':str(m_width) + ',' + str(m_height),
                'imageSR':outSR,
                'format':'tiff'
            }
            print(parm['bbox'])
            response=requests.get(qry,parm)
            if response.status_code == 200:
                img_url=response.json()['href']
                outname=out_prefix + str(tile) + '.tif'
                urllib.request.urlretrieve(img_url, outname)
                rs_lst.append(Raster(outname))
                tile+=1

    return rs_lst

def get_feature_layer_data(url, geo='',qry='1=1',layer=0):
  '''
  gets a geodataframe from a Feature Service given the url and optionally a bounding geometry and where clause

  url=(string) base url for the feature service
  geo=(object) a bounding box string, shapely polygon, geodataframe, or geoseries. string and shapely polygon objects are assumed to be in the same coordinate system as the feature service
  qry=(string) where clause used to subset the data
  layer= (int) the of the feature service to extract

  return a geodataframe of features
  '''
  s_info=requests.get(url+'?f=pjson').json()
  srn=s_info['spatialReference']['wkid']
  sr='EPSG:'+str(srn)
  if isinstance(geo,gpd.GeoDataFrame):
    geo = (geo.to_crs(sr)).total_bounds
  elif isinstance(geo,gpd.GeoSeries):
    geo = (geo.to_crs(sr)).total_bounds
  elif isinstance(geo,shapely.geometry.Polygon):
    geo = geo.bounds
  else:
    pass
  geo=','.join(np.array(geo).astype(str))
  url1=url+'/'+str(layer)
  l_info=requests.get(url1 + '?f=pjson').json()
  maxrcn=l_info['maxRecordCount']
  if maxrcn>100: maxrcn=100 #used to subset ids so query is not so long
  url2 = url1+'/query?'
  o_info=requests.get(url2,{'where': qry,'geometry':geo,'geometryType': 'esriGeometryEnvelope','returnIdsOnly':'True','f': 'pjson'}).json()
  oid_name=o_info['objectIdFieldName']
  oids=o_info['objectIds']
  numrec=len(oids)
  fslist = []
  for i in range(0, numrec, maxrcn):
    torec = i + (maxrcn-1)
    if torec > numrec:
      torec = numrec

    objectIds = oids[i:torec]
    idstr=oid_name + ' in (' + str(objectIds)[1:-1]+')'
    #note that parameter values depend on the service
    prm={
        'where': idstr,
        'outFields': '*',
        'returnGeometry': 'true',
        'outSR':srn,
        'f':'pgeojson',
    }
    ftrs=requests.get(url2,prm).json()['features']
    fslist.append(gpd.GeoDataFrame.from_features(ftrs,crs=sr))

  return gpd.pd.concat(fslist)



## Get DEM data from arcgis rest service for area of interest

In [ ]:
url_dem=r'https://elevation3d.arcgis.com/arcgis/rest/services/WorldElevation3D/Terrain3D/ImageServer'
dem_lst=get_image_service_data(url_dem,ply,'DEM',30,5070)

## Get EVT data form Landfire for area of interest

In [ ]:
url_evt=r'https://lfps.usgs.gov/arcgis/rest/services/Landfire_LF230/US_230EVT/ImageServer' #EVT url
evt_lst=get_image_service_data(url_evt,ply,'EVT',30,5070)

## POD data

In [ ]:
url = r'https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/Nat_PODs_Public/FeatureServer'
pods=get_feature_layer_data(url,gpd.GeoDataFrame(geometry=ply,crs=ply.crs),layer=1)

## Visualize the data

Look at the third DEM returned

In [ ]:
dem_lst[2].plot(cmap='terrain',figsize=(15,15))

Look at EVT raster

In [ ]:
evt_lst[0].plot(figsize=(15,15))

Look at PODs in folium

In [ ]:
pods.explore()